<a href="https://colab.research.google.com/github/simulate111/Climatic_Data/blob/main/Oslo_Norway_meteorological_data_Frost(2015_2024).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import requests
import pandas as pd
import time
from datetime import datetime

client_id = '95b97914-f898-42d6-b1de-b1557d61357d'
STATION_ID = 'SN18700'
YEARS = range(2015, 2025)
OUTPUT_FILE = "Oslo_10Yr_10Minute_Average.csv"

# Basic elements (most reliable for Frost API)
tasks = [
    {"element": "surface_downwelling_shortwave_flux_in_air", "name": "GHI_Wm2"},
    {"element": "air_temperature", "name": "Temperature_C"},
    {"element": "wind_speed", "name": "Wind_Speed_ms"}
]

all_years_data = []

for year in YEARS:
    print(f"\n--- Fetching Year: {year} ---")
    yearly_task_dfs = []

    for task in tasks:
        print(f"  > Downloading {task['name']}...")

        params = {
            'sources': STATION_ID,
            'elements': task['element'],
            'referencetime': f"{year}-01-01/{year+1}-01-01"
        }

        try:
            r = requests.get('https://frost.met.no/observations/v0.jsonld', params, auth=(client_id, ''))

            if r.status_code == 200:
                data = r.json().get('data', [])
                rows = []
                for item in data:
                    val = item['observations'][0]['value']
                    rows.append({'Time': item['referenceTime'], task['name']: val})

                if rows:
                    df = pd.DataFrame(rows)
                    df['Time'] = pd.to_datetime(df['Time'])
                    df = df.set_index('Time').sort_index()

                    # This line is key: it forces the raw data into a 10-min grid
                    df = df.resample('10min').mean().interpolate(limit=3)
                    yearly_task_dfs.append(df)
            else:
                print(f"    [!] Skip: API returned {r.status_code} - {r.text}")

        except Exception as e:
            print(f"    Error: {e}")
        time.sleep(0.2)

    if yearly_task_dfs:
        df_year = pd.concat(yearly_task_dfs, axis=1)
        all_years_data.append(df_year.reset_index())

# (Keep the rest of your AGGREGATION logic the same as before)
if all_years_data:
    print(f"\nProcessing 10-year average...")
    master_df = pd.concat(all_years_data)
    master_df = master_df[~((master_df['Time'].dt.month == 2) & (master_df['Time'].dt.day == 29))]
    master_df['Month'] = master_df['Time'].dt.month
    master_df['Day'] = master_df['Time'].dt.day
    master_df['Hour'] = master_df['Time'].dt.hour
    master_df['Minute'] = master_df['Time'].dt.minute

    climate_avg = master_df.groupby(['Month', 'Day', 'Hour', 'Minute']).mean(numeric_only=True).reset_index()
    climate_avg.to_csv(OUTPUT_FILE, index=False)
    print(f"Success! Saved to {OUTPUT_FILE}")


--- Fetching Year: 2015 ---
  > Downloading GHI_Wm2...
    [!] Skip: API returned 412 - {
  "@context" : "https://frost.met.no/schema",
  "@type" : "ErrorResponse",
  "apiVersion" : "v0",
  "license" : "https://creativecommons.org/licenses/by/3.0/no/",
  "createdAt" : "2025-12-21T11:10:50Z",
  "queryTime" : 0.278,
  "currentLink" : "https://frost.met.no/observations/v0.jsonld?sources=SN18700&elements=surface_downwelling_shortwave_flux_in_air&referencetime=2015-01-01%2F2016-01-01",
  "error" : {
    "code" : 412,
    "message" : "412",
    "reason" : "No time series found for this combination of parameters, check /observations/availableTimeSeries for more information."
  }
}
  > Downloading Temperature_C...
  > Downloading Wind_Speed_ms...

--- Fetching Year: 2016 ---
  > Downloading GHI_Wm2...
    [!] Skip: API returned 412 - {
  "@context" : "https://frost.met.no/schema",
  "@type" : "ErrorResponse",
  "apiVersion" : "v0",
  "license" : "https://creativecommons.org/licenses/by/3.0/no